In [2]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.utils import get_device

import datetime

/opt/conda/envs/detection/lib/python3.7/site-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '


In [3]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')

root='../../dataset/'

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
#cfg.data.train.ann_file = root + 'pseudo_concat_new.json'
cfg.data.train.ann_file = root + 'train_MultiStfKFold.json' # train json 정보

cfg.data.val.classes = classes
cfg.data.val.img_prefix = root
cfg.data.val.ann_file = root + 'val_MultiStfKFold.json'

'''
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)

cfg.data.train.pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(type='Resize', img_scale=(1333, 800), keep_ratio=True),
    dict(type = 'Albu',
         transforms = [dict(type='Rotate', p=1, rotate_method='ellipse', limit=15)],
         #transforms = [dict(type='RandomRotate90', p=1)],
         #transforms = [],
         bbox_params = dict(type='BboxParams', format='pascal_voc', label_fields=['gt_labels']),
         keymap = {'img': 'image', 'gt_bboxes': 'bboxes'}),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])]
'''

cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize

now = (datetime.datetime.now().replace(microsecond=0) + datetime.timedelta(hours=9)).strftime("%m-%d %H:%M")
cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='WandbLoggerHook',
         init_kwargs={
            'project': 'trash_detection',
            'entity': 'doyunkim',
            'name' : f'{cfg.model.type}_{now}_dyk',
            'tags': [cfg.model.backbone.type, cfg.model.neck.type]
            },
         interval=10)]

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize
#cfg.data.val.pipeline = cfg.data.test.pipeline

cfg.data.samples_per_gpu = 4

cfg.seed = 2022
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/faster_rcnn_r50_fpn_1x_trash'+now

cfg.model.roi_head.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)
cfg.device = get_device()

In [8]:
#cfg.runner.max_epochs = 50
#cfg.data.train.classes
#cfg.lr_config.step = [40,46]
#cfg.optimizer= {'type': 'Adam', 'lr': 0.00002, 'weight_decay': 0.0001}
#cfg.workflow = [('train',1),('val',1)]
cfg.resume_from

In [56]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.49s)
creating index...
index created!


In [57]:
# dataset 확인
datasets[0]


CocoDataset Train dataset with number of images 8590, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 4734  | 1 [Paper]     | 8388  | 2 [Paper pack]  | 1382  | 3 [Metal]   | 1329  | 4 [Glass]    | 1394  |
| 5 [Plastic]       | 3956  | 6 [Styrofoam] | 1752  | 7 [Plastic bag] | 7708  | 8 [Battery] | 224   | 9 [Clothing] | 671   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [58]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

2022-11-29 01:28:44,689 - mmcv - INFO - initialize ResNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet50'}
2022-11-29 01:28:44,692 - mmcv - INFO - load model from: torchvision://resnet50
2022-11-29 01:28:44,696 - mmcv - INFO - load checkpoint from torchvision path: torchvision://resnet50
2022-11-29 01:28:44,914 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

2022-11-29 01:28:44,942 - mmcv - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2022-11-29 01:28:44,975 - mmcv - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2022-11-29 01:28:44,986 - mmcv - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'distribution': 'uniform', 'override'

In [59]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

2022-11-29 01:28:45,978 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2022-11-29 01:28:46,007 - mmdet - INFO - Start running, host: root@a78b80fabad9, work_dir: /opt/ml/baseline/mmdetection/work_dirs/faster_rcnn_r50_fpn_1x_trash11-29 10:24
2022-11-29 01:28:46,008 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
bef

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


learning_rate,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇▇██████████████████
momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/acc,▁████████▇▇██▇▇██▇██▇▇██▇▇▇▇█▇█▇█▇▇▇██▇█
train/grad_norm,█▂▂▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,█▃▂▁▁▁▁▁▁▂▂▂▂▂▂▂▁▂▁▁▂▂▂▁▂▂▂▂▁▂▁▂▁▂▂▂▁▁▂▁
train/loss_bbox,▁▃▅▅▅▆▅▆▅▇▇▆▆▇▇▆▆▇▅▅▇▇▆▅█▇▆▇▅█▅▆▆▆▇▆▅▆▇▅
train/loss_cls,█▂▁▁▁▁▁▁▁▁▁▁▁▂▂▁▁▁▁▁▁▂▁▁▂▁▁▁▁▂▁▁▁▁▁▁▁▁▂▁
train/loss_rpn_bbox,█▄▅▃▁▅▂▃▃▆▅▃▅▅▅▄▃▆▃▂▅▅▅▃▆▄▅▅▂▆▃▄▃▃▄▄▃▄▇▃
train/loss_rpn_cls,█▆▂▁▁▁▁▁▁▂▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁
learning_rate,0.02
momentum,0.9


2022-11-29 01:29:11,715 - mmdet - INFO - Epoch [1][50/2148]	lr: 1.978e-03, eta: 8:10:28, time: 0.274, data_time: 0.052, memory: 2572, loss_rpn_cls: 0.3985, loss_rpn_bbox: 0.0383, loss_cls: 0.4997, acc: 91.6592, loss_bbox: 0.0894, loss: 1.0258, grad_norm: 6.7130
2022-11-29 01:29:23,443 - mmdet - INFO - Epoch [1][100/2148]	lr: 3.976e-03, eta: 7:34:51, time: 0.235, data_time: 0.007, memory: 2583, loss_rpn_cls: 0.1220, loss_rpn_bbox: 0.0389, loss_cls: 0.3141, acc: 94.0566, loss_bbox: 0.2358, loss: 0.7108, grad_norm: 2.7661
2022-11-29 01:29:35,073 - mmdet - INFO - Epoch [1][150/2148]	lr: 5.974e-03, eta: 7:21:41, time: 0.233, data_time: 0.006, memory: 2583, loss_rpn_cls: 0.1045, loss_rpn_bbox: 0.0348, loss_cls: 0.3121, acc: 93.6162, loss_bbox: 0.2553, loss: 0.7066, grad_norm: 2.1715
2022-11-29 01:29:46,731 - mmdet - INFO - Epoch [1][200/2148]	lr: 7.972e-03, eta: 7:15:15, time: 0.233, data_time: 0.006, memory: 2583, loss_rpn_cls: 0.1009, loss_rpn_bbox: 0.0364, loss_cls: 0.3340, acc: 93.1123, 

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 976/976, 21.5 task/s, elapsed: 45s, ETA:     0s

...

Loading and preparing results...
DONE (t=0.05s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=5.95s).
Accumulating evaluation results...


2022-11-29 05:58:29,762 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.151
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.325
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.113
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.003
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.077
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.176
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.356
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.356
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.356
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.011
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.160
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000

DONE (t=1.71s).


2022-11-29 05:59:05,622 - mmdet - INFO - Epoch [13][50/2148]	lr: 2.000e-02, eta: 12:46:24, time: 0.716, data_time: 0.064, memory: 2585, loss_rpn_cls: 0.0249, loss_rpn_bbox: 0.0240, loss_cls: 0.2115, acc: 92.9961, loss_bbox: 0.2077, loss: 0.4682, grad_norm: 1.5184
2022-11-29 05:59:37,719 - mmdet - INFO - Epoch [13][100/2148]	lr: 2.000e-02, eta: 12:46:08, time: 0.642, data_time: 0.011, memory: 2585, loss_rpn_cls: 0.0235, loss_rpn_bbox: 0.0245, loss_cls: 0.1929, acc: 93.3643, loss_bbox: 0.1890, loss: 0.4298, grad_norm: 1.4844
2022-11-29 06:00:10,451 - mmdet - INFO - Epoch [13][150/2148]	lr: 2.000e-02, eta: 12:45:54, time: 0.655, data_time: 0.010, memory: 2585, loss_rpn_cls: 0.0234, loss_rpn_bbox: 0.0245, loss_cls: 0.2096, acc: 92.7588, loss_bbox: 0.2073, loss: 0.4647, grad_norm: 1.5411
2022-11-29 06:00:42,914 - mmdet - INFO - Epoch [13][200/2148]	lr: 2.000e-02, eta: 12:45:39, time: 0.649, data_time: 0.011, memory: 2585, loss_rpn_cls: 0.0201, loss_rpn_bbox: 0.0200, loss_cls: 0.2161, acc: 9

KeyboardInterrupt: 